In [ ]:
import pandas as pd
from merge_cedulas import fill_missing_info  # Importar función del repositorio
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

filepath = "C:/Users/Lenovo/Visual/ARI/ARI/Proyecto Integrador/prep-data/pcd_1211.csv"

# Cargar los datos (puedes reemplazar 'tu_archivo.csv' por el nombre de tu archivo de datos)
df = pd.read_csv(filepath)

# Unificar las columnas 'tipo de discapacidad' y 'Describa aqui el diagnostico medico'
df['discapacidad_combinada'] = df['tipo de discapacidad'].fillna('') + ' ' + df['Describa aqui el diagnostico medico'].fillna('')

# Limpiar los espacios adicionales
df['discapacidad_combinada'] = df['discapacidad_combinada'].str.strip()

# Completar información faltante usando la función del script 'merge_cedulas.py'
df = fill_missing_info(df, 'Fecha de la encuesta')

# Aplicar TF-IDF para vectorizar la columna 'discapacidad_combinada'
tfidf_vectorizer = TfidfVectorizer(stop_words='spanish')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['discapacidad_combinada'])

# Aplicar KMeans para agrupar pueblos con características similares respecto a las discapacidades
num_clusters = 5  # Puedes ajustar el número de clusters según el análisis
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

# Añadir la etiqueta del cluster al dataframe
df['cluster'] = kmeans.labels_

# Visualizar la distribución de los clusters
plt.figure(figsize=(10, 6))
plt.hist(df['cluster'], bins=num_clusters, edgecolor='black')
plt.xlabel('Cluster')
plt.ylabel('Número de Pueblos')
plt.title('Distribución de Clusters por Tipo de Discapacidad y Diagnóstico Médico')
plt.show()

# Guardar el nuevo dataset con la columna combinada y los clusters
df.to_csv('datos_con_discapacidad_clusters.csv', index=False)
